https://github.com/TianzhongSong/C3D-keras

In [1]:
from models import c3d_model # models.py 다운받기
from keras.optimizers import SGD,Adam
from keras.utils import np_utils
from schedules import onetenth_4_8_12 # schedules.py, callbacks.py 다운받기
import numpy as np
import random
import cv2
import os
import random
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import json # 구글 드라이브에 있는 metadata.json 파일 로컬에 다운받기
import util.collection_util as cu # 깃허브 VideoSummarizer에 있는 'util'폴더 그대로 로컬에 다운받기

def plot_history(history, result_dir):
    plt.plot(history.history['acc'], marker='.')
    plt.plot(history.history['val_acc'], marker='.')
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['acc', 'val_acc'], loc='lower right')
    plt.savefig(os.path.join(result_dir, 'model_accuracy.png'))
    plt.close()

    plt.plot(history.history['loss'], marker='.')
    plt.plot(history.history['val_loss'], marker='.')
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend(['loss', 'val_loss'], loc='upper right')
    plt.savefig(os.path.join(result_dir, 'model_loss.png'))
    plt.close()


def save_history(history, result_dir):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(os.path.join(result_dir, 'result.txt'), 'w') as fp:
        fp.write('epoch\tloss\tacc\tval_loss\tval_acc\n')
        for i in range(nb_epoch):
            fp.write('{}\t{}\t{}\t{}\t{}\n'.format(
                i, loss[i], acc[i], val_loss[i], val_acc[i]))
        fp.close()

Using TensorFlow backend.


In [2]:
# 본 함수들

def process_batch(dataset_dir, num):

    # 하나의 큰 배치가 될 5차원 영행렬
    batch = np.zeros((num,15,64,64,3),dtype='float16')

    # 0,1 라벨이 담길 1차원 영행렬
    labels = np.zeros(num,dtype='int')

    #dataset_dir = '20200205_KT_GEN_1_pkl'
    path = os.listdir(dataset_dir)

    for i in range(num):
        labels[i] = path[i][10] # 피클 파일명에서 레이블을 뽑음. 0또는 1
        segment_path  = os.path.join(dataset_dir, path[i])
        segment = cu.load(segment_path)
        video = segment['video']
        for j, image in enumerate(video):
            batch[i][j][:][:][:] = image
            
    return batch, labels

# 꼭 필요한가? 상황에 맞춰 바꿔볼 수 있을듯.
def preprocess(inputs):
    inputs[..., 0] -= 99.9
    inputs[..., 1] -= 92.1
    inputs[..., 2] -= 82.6
    inputs[..., 0] /= 65.8
    inputs[..., 1] /= 62.3
    inputs[..., 2] /= 60.3
    # inputs /=255.
    # inputs -= 0.5
    # inputs *=2.
    return inputs

def generator_train_batch(dataset_dir,num_classes,batch_size):
    with open(os.path.join('metadata.json')) as f:
        metadata = json.load(f)
            
    # 현재 '20200205_KT_GEN_1'이 938개의 피클로 이루어졌고 각각 15장의 사진이 들어가있음
    # 938 : 예를 들어 dataset_dir='20200205_KT_GEN_1_pkl'이라는 폴더 안에 938개의 pkl이 있는것
    # num = metadata['segment_counts']['20200205_KT_GEN_1']로 알 수 있으므로 폴더명(dataset_dir)에서 _pkl만 공백으로 대체

    num = metadata['segment_counts'][dataset_dir.replace('_pkl','')]
    while True:
        for i in range(num): # batch_size = 1, 이 상태에서 [a:b]는 배치 1개씩 차례대로 보겠다는 것
            a = i * batch_size 
            b = (i+1) * batch_size
            
            # process_batch(dataset_dir)[0][a:b].shape = (1, 15, 64, 64, 3), 즉 피클 1개에 있는 15장의 사진이 배치 하나가 됨
            # 따라서 batch_size = 1, 이 상태에서 [a:b]의 뜻은 배치 1개씩 차례대로 보겠다는 것
            
            x_train = process_batch(dataset_dir,num)[0][a:b] # process_batch의 첫번째 리턴값 batch의 [a:b]
            x_labels = process_batch(dataset_dir,num)[1][a:b] # process_batch의 두번째 리턴값 label의 [a:b]
            x = preprocess(x_train)
            y = np_utils.to_categorical(np.array(x_labels), num_classes) # y.shape = (938, 2)
            x = np.transpose(x, (0,2,3,1,4)) # x.shape = (1, 64, 15, 64, 3)로 재배치, 왜??
            yield x, y
            
def generator_val_batch(dataset_dir,num_classes,batch_size):
    with open(os.path.join('metadata.json')) as f:
        metadata = json.load(f)
    num = metadata['segment_counts'][dataset_dir.replace('_pkl','')]
    while True:
        for i in range(num):
            a = i * batch_size
            b = (i+1) * batch_size
            y_test = process_batch(dataset_dir,num)[0][a:b]
            y_labels = process_batch(dataset_dir,num)[1][a:b]
            x = preprocess(y_test)
            x = np.transpose(x,(0,2,3,1,4))
            y = np_utils.to_categorical(np.array(y_labels), num_classes)
            yield x, y

In [3]:
def main():
    train_dir = '20200205_KT_GEN_1_pkl' # train 세트가 될 pkl파일들이 모인 폴더 
    test_dir = '20200205_T1_DWG_1_pkl'  # test 세트가 될 pkl파일들이 모인 폴더
    
    length_train = len(os.listdir(train_dir)) # train_dir안에 있는 pkl 파일 갯수
    length_test = len(os.listdir(test_dir))

    num_classes = 2 # 0,1 클래스 두 종류
    batch_size = 1
    epochs = 8

    model = c3d_model()
    lr = 0.001
    sgd = SGD(lr=lr, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.summary()
    history = model.fit_generator(generator_train_batch(train_dir,num_classes,batch_size),
                                  steps_per_epoch=length_train // batch_size,
                                  epochs=epochs,
                                  callbacks=[onetenth_4_8_12(lr)],
                                  validation_data=generator_val_batch(test_dir,num_classes,batch_size),
                                  validation_steps=length_test // batch_size,
                                  verbose=1)
    if not os.path.exists('results/'):
        os.mkdir('results/')
    plot_history(history, 'results/')
    save_history(history, 'results/')
    model.save_weights('results/weights_c3d.h5')


if __name__ == '__main__':
    main()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 15, 3)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 64, 64, 15, 64)    5248      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 32, 32, 15, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 32, 32, 15, 128)   221312    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 16, 16, 8, 128)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 16, 16, 8, 128)    442496    
____________________________________________

MemoryError: Unable to allocate array with shape (1079, 15, 64, 64, 3) and data type float16